In [3]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
db.reset()

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code):
    abi = ''
    account_name = 'test11111111'
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)

# s2n, n2s, N

s2n把字符串转成uint64_t类型， n2s, N的功能一样，都是把uint64_t类型转换成字符串，
注意这里的N是一个函数，参数是一个字符串，不要和C++里面的宏`N`混淆了，C++里的`N`是不用带""的


In [36]:
code = '''
def apply(receiver, code, action):
    print(receiver, code, action)
    print(n2s(receiver), n2s(code), n2s(action))
    print(n2s(receiver) == 'test11111111')
    print(N('test11111111') == receiver)
'''
name = 'test11111111'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

14605613963703501328 14605613963703501328 13960267442438537216
test11111111 test11111111 sayhello
True
True



# current_receiver
获取当前的receiver

In [38]:
code = '''
import struct
def apply(receiver, code, action):
    print(receiver == current_receiver())
    print(n2s(current_receiver()))
    
'''
name = 'test11111111'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

True
test11111111



# read_action_data

获取action的data字段

In [40]:
code = '''
def apply(receiver, code, action):
    print(read_action_data())
'''

abi = ''
account_name = 'test11111111'
publish_contract(account_name, code, abi)
try:
    r = eosapi.push_action(account_name, 'sayhello', b'goodbye,world', {account_name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)


b'goodbye,world'

283


# is_account

In [46]:
code = '''
def apply(receiver, code, action):
    print(is_account(receiver))
    print(is_account('test11111111'))
'''

run_test_code(code)

True
True



# publication_time

In [5]:
code = '''
def apply(receiver, code, action):
    print(publication_time())
'''

run_test_code(code)

1559921419500000
1559921419500000
1559921419



In [23]:
import time
local_time = time.localtime(1559901448500000/1e6)
time.strftime('%a, %d %b %Y %H:%M:%S GMT', local_time)

Sat, 08 Jun 2019 00:50:07 GMT
Sat, 08 Jun 2019 00:50:12 GMT


# has_auth

In [49]:
code = '''
def apply(receiver, code, action):
    print(has_auth(receiver))
    print(has_auth('test11111111'))
    print(has_auth('eosio'))
'''

run_test_code(code)

True
True
False



# require_auth/require_auth2

In [53]:
code = '''
def apply(receiver, code, action):
#    require_auth('eosio')
    require_auth2('test11111111', 'owner')
    print('done!')
'''

abi = ''
account_name = 'test11111111'
publish_contract(account_name, code, abi)
try:
    r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'owner'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

done!



# publication_time/current_time/now

```C++
uint64_t  publication_time() {
   return static_cast<uint64_t>( ctx().trx_context.published.time_since_epoch().count() );
}

uint64_t  current_time() {
   return static_cast<uint64_t>( ctx().control.pending_block_time().time_since_epoch().count() );
}

uint32_t  now() {
   return (uint32_t)( current_time() / 1000000 );
}

```

In [35]:
code = '''
def apply(receiver, code, action):
    print(publication_time())
    print(current_time())
    print(now())
'''

abi = ''
account_name = 'test11111111'
publish_contract(account_name, code, abi)
try:
    r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'owner'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

1559965875500000
1559965875500000
1559965875



# eosio_assert/eosio_assert_code

In [43]:
code = '''
def apply(receiver, code, action):
    eosio_assert(False, 'oops!')
    eosio_assert_code(receiver!=code, 111)
'''

abi = ''
account_name = 'test11111111'
publish_contract(account_name, code, abi)
try:
    r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'owner'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

INFO:pyeoskit.http_client:non ok response: 500


API returned status code: 500 {"code":500,"message":"Internal Service Error","error":{"code":3050003,"name":"eosio_assert_message_exception","what":"eosio_assert_message assertion failure","details":[{"message":"assertion failure with message: oops!","file":"system.cpp","line_number":18,"method":"eosio_assert"}]}}
